In [1]:
total_file = './data/Result_TFIDF.csv'

source_folder = './data/class/train'
destination_folder = './data/class/model'

In [2]:
label_mapping ={
    '傷害' : 0,
    '侵占' : 1,
    '公共危險' : 2,
    '過失傷害' : 3,
    '妨害自由' : 4,
    '妨害公務' : 5,
    '竊盜' : 6,
    '妨害名譽' : 7,
    '妨害性自主' : 8,
    '偽造文書' : 9,
    '搶奪' : 10,
    '毀損' : 11,
    '妨害風化' : 12,
    '毒品危害防制' : 13,
    '妨害性自主罪' : 14,
    '強盜' : 15,
    '著作權法' : 16,
    '過失致死' : 17,
    '詐欺' : 17,
    '賭博' : 17,
    '槍砲彈藥刀械管制' : 17,
    '貪污' : 17,
    '殺人未遂' : 17,
    '醫師法' : 17,
    '藥事法' : 17,
    '偽證' : 17,
    '殺人' : 17
}

In [3]:
# Libraries

import matplotlib.pyplot as plt
import pandas as pd
import torch
import os, json


# Preliminaries

from torchtext.data import Field, TabularDataset, BucketIterator, Iterator

# Models

import torch.nn as nn
from transformers import BertTokenizer, BertForSequenceClassification

# Training

import torch.optim as optim

# Evaluation

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [4]:
dffull = pd.read_csv(total_file, index_col=0)
dffull['reason'] = dffull['reason'].str.replace('等','')
dffull['reason'] = dffull['reason'].str.replace("條例","")
dffull['reason'] = dffull['reason'].str.replace("違反","")
dffull['text'] = dffull['Situation']
dffull['label'] = dffull['reason'].str.strip().map(label_mapping)

dffull = dffull.rename(columns={"mainText": "title", "Situation": "titletext"})
dffull = dffull[["no", "label", "title", "text", "titletext"]]
dffull = dffull.dropna()
dffull.head(3)

,no,label,title,text,titletext
id,,,,,
0,"106,易,1093",5,邱建豪犯妨害公務執行罪，累犯，處拘役拾日，如易科罰金，以新臺幣壹仟元折算壹日。,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...,邱建豪於民國106年5月1日下午5時15分許，在桃園市○○區○○路000號桃園市政府警察局桃...
1,"100,中交簡,1002",2,王炎權服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾日，如易科罰金，以新臺幣壹仟元折算壹日。,王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉○○路○段2...,王炎權於民國100年4月8日晚間9時30分許起至翌日凌晨3時許止，在彰化縣芬園鄉○○路○段2...
2,"100,中交簡,100",2,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...


In [5]:
dffull.loc[:,'label'].value_counts()

0     220
1     210
2     206
3     198
4     191
5     184
6     178
7     169
8      91
9      79
10     75
11     69
12     68
13     66
15     63
14     63
16     61
17     36
Name: label, dtype: int64

In [6]:
dffull.isna().sum()

no           0
label        0
title        0
text         0
titletext    0
dtype: int64

In [7]:
##拆分df
dftrain=dffull.sample(frac=0.6)#按0.6比例随机采样
dftrain.head(3)

,no,label,title,text,titletext
id,,,,,
37,"100,中簡,128",6,張明杉犯竊盜罪，處拘役參拾日，如易科罰金，以新臺幣壹仟元折算壹日；又犯竊盜罪，處拘役貳拾日，...,於民國99年4月13日9時28分許、同年月15日10時13分許及同年月16日9時49分許，在...,於民國99年4月13日9時28分許、同年月15日10時13分許及同年月16日9時49分許，在...
1704,"105,訴,221",9,蕭萬呈無罪。,被告蕭萬呈於民國95年5月25日起至99年5月25日止，在「台灣路跑馬拉松協會」（下稱台灣路...,被告蕭萬呈於民國95年5月25日起至99年5月25日止，在「台灣路跑馬拉松協會」（下稱台灣路...
27,"100,中簡,1183",6,郭榮欽竊盜，處罰金新臺幣貳仟元，如易服勞役，以新臺幣壹仟元折算壹日。,於民國100年4月2日下午3時15分許，在臺中市○里區○○路○段710號「大買家量販店」內，...,於民國100年4月2日下午3時15分許，在臺中市○里區○○路○段710號「大買家量販店」內，...


In [8]:
dftrain.loc[:,'label'].value_counts()

1     133
0     125
2     121
3     118
4     118
6     116
5     115
7      98
8      50
10     46
12     46
9      44
14     40
15     40
16     38
13     36
11     32
17     20
Name: label, dtype: int64

In [9]:
tmp = dffull[~dffull['no'].isin(dftrain['no'])]
tmp.count()

no           870
label        870
title        870
text         870
titletext    870
dtype: int64

In [10]:
##拆分df
dfvalid=tmp.sample(frac=0.5)#按0.6比例随机采样
dfvalid.head(3)

,no,label,title,text,titletext
id,,,,,
39,"100,中簡,131",6,黃添益犯竊盜罪，累犯，處有期徒刑陸月，如易科罰金，以新台幣壹仟元折算壹日。,於99年12月15日中午12時許至同日23時45分前間之某時，在臺中市○○路140號前，以自...,於99年12月15日中午12時許至同日23時45分前間之某時，在臺中市○○路140號前，以自...
645,"105,交易,216",3,本件公訴不受理。,被告許信堂係車牌號碼0000-00號自用小客車之所有人，詎其本應注意車輛停放位置，且明知在繪...,被告許信堂係車牌號碼0000-00號自用小客車之所有人，詎其本應注意車輛停放位置，且明知在繪...
2256,"106,訴,12",15,林浦辰犯竊盜罪，處有期徒刑貳月，如易科罰金，以新臺幣壹仟元折算壹日，並應於刑之執行完畢或赦免...,林浦辰意圖為自己不法之所有，於民國105年10月30日20時30分許，在臺北市○○區○○路0...,林浦辰意圖為自己不法之所有，於民國105年10月30日20時30分許，在臺北市○○區○○路0...


In [11]:
dftest = tmp[~tmp['no'].isin(dfvalid['no'])]
tmp.count()

no           870
label        870
title        870
text         870
titletext    870
dtype: int64

In [12]:
dftest.head(3)

,no,label,title,text,titletext
id,,,,,
2,"100,中交簡,100",2,林耀明服用酒類，不能安全駕駛動力交通工具而駕駛，處拘役伍拾伍日，如易科罰金，以新臺幣壹仟元折...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...,林耀明於民國99年10月22日下午4時許起至5時許止，在臺中市○○區○○路與豐樂路交岔路口附...
7,"100,中簡,1023",12,許其昌共同意圖使女子與他人為性交之行為，而媒介以營利，處有期徒刑肆月，如易科罰金，以新臺幣壹...,許其昌係址設臺中市○區○○○路374號「左岸國際美容生活館」(下稱左岸美容館)之負責人，而李...,許其昌係址設臺中市○區○○○路374號「左岸國際美容生活館」(下稱左岸美容館)之負責人，而李...
9,"100,中簡,1040",6,詹金龍犯竊盜罪，處罰金新臺幣壹萬元，如易服勞役，以新臺幣壹仟元折算壹日。,於民國100年3月25日5時30分許，在臺中市南區永興5巷42號前，徒手竊取劉清智所有放置該...,於民國100年3月25日5時30分許，在臺中市南區永興5巷42號前，徒手竊取劉清智所有放置該...


In [13]:
dftrain = dftrain.drop(columns=['no'])
dfvalid = dfvalid.drop(columns=['no'])
dftest = dftest.drop(columns=['no'])
dftrain.to_csv(source_folder + '/train.csv', index=0)
dfvalid.to_csv(source_folder + '/valid.csv', index=0)
dftest.to_csv(source_folder + '/test.csv', index=0)

In [14]:
#start class

In [15]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [16]:
tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')

# Model parameter
MAX_SEQ_LEN = 512
PAD_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.pad_token)
UNK_INDEX = tokenizer.convert_tokens_to_ids(tokenizer.unk_token)

# Fields

label_field = Field(sequential=False, use_vocab=False, batch_first=True, dtype=torch.float)
text_field = Field(use_vocab=False, tokenize=tokenizer.encode, lower=False, include_lengths=False, batch_first=True,
                   fix_length=MAX_SEQ_LEN, pad_token=PAD_INDEX, unk_token=UNK_INDEX)
fields = [('label', label_field), ('title', text_field), ('text', text_field), ('titletext', text_field)]

# TabularDataset

train, valid, test = TabularDataset.splits(path=source_folder, train='train.csv', validation='valid.csv',
                                           test='test.csv', format='CSV', fields=fields, skip_header=True)

# Iterators

train_iter = BucketIterator(train, batch_size=2, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
valid_iter = BucketIterator(valid, batch_size=2, sort_key=lambda x: len(x.text),
                            device=device, train=True, sort=True, sort_within_batch=True)
test_iter = Iterator(test, batch_size=2, device=device, train=False, shuffle=False, sort=False)

In [17]:
class BERT(nn.Module):

    def __init__(self):
        super(BERT, self).__init__()

        options_name = "bert-base-chinese"
        self.encoder = BertForSequenceClassification.from_pretrained(options_name)

    def forward(self, text, label):
        loss, text_fea = self.encoder(text, labels=label)[:2]

        return loss, text_fea

In [18]:
# Save and Load Functions

def save_checkpoint(save_path, model, valid_loss):

    if save_path == None:
        return

    state_dict = {'model_state_dict': model.state_dict(),
                  'valid_loss': valid_loss}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')

def load_checkpoint(load_path, model):

    if load_path==None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    model.load_state_dict(state_dict['model_state_dict'])
    return state_dict['valid_loss']


def save_metrics(save_path, train_loss_list, valid_loss_list, global_steps_list):

    if save_path == None:
        return

    state_dict = {'train_loss_list': train_loss_list,
                  'valid_loss_list': valid_loss_list,
                  'global_steps_list': global_steps_list}

    torch.save(state_dict, save_path)
    print(f'Model saved to ==> {save_path}')


def load_metrics(load_path):

    if load_path==None:
        return

    state_dict = torch.load(load_path, map_location=device)
    print(f'Model loaded from <== {load_path}')

    return state_dict['train_loss_list'], state_dict['valid_loss_list'], state_dict['global_steps_list']

In [19]:
# Training Function

def train(model,
          optimizer,
          criterion = nn.MSELoss(),
          train_loader = train_iter,
          valid_loader = valid_iter,
          num_epochs = 5,
          eval_every = len(train_iter) // 2,
          file_path = destination_folder,
          best_valid_loss = float("Inf")):

    # initialize running values
    running_loss = 0.0
    valid_running_loss = 0.0
    global_step = 0
    train_loss_list = []
    valid_loss_list = []
    global_steps_list = []

    # training loop
    model.train()
    for epoch in range(num_epochs):
        for (labels, title, text, titletext), _ in train_loader:
            labels = labels.type(torch.LongTensor)
            labels = labels.to(device)
            titletext = titletext.type(torch.LongTensor)
            titletext = titletext.to(device)
            output = model(titletext, labels)
            loss, _ = output

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # update running values
            running_loss += loss.item()
            global_step += 1

            # evaluation step
            if global_step % eval_every == 0:
                model.eval()
                with torch.no_grad():

                    # validation loop
                    for (labels, title, text, titletext), _ in valid_loader:
                        labels = labels.type(torch.LongTensor)
                        labels = labels.to(device)
                        titletext = titletext.type(torch.LongTensor)
                        titletext = titletext.to(device)
                        output = model(titletext, labels)
                        loss, _ = output

                        valid_running_loss += loss.item()

                # evaluation
                average_train_loss = running_loss / eval_every
                average_valid_loss = valid_running_loss / len(valid_loader)
                train_loss_list.append(average_train_loss)
                valid_loss_list.append(average_valid_loss)
                global_steps_list.append(global_step)

                # resetting running values
                running_loss = 0.0
                valid_running_loss = 0.0
                model.train()

                # print progress
                print('Epoch [{}/{}], Step [{}/{}], Train Loss: {:.4f}, Valid Loss: {:.4f}'
                      .format(epoch+1, num_epochs, global_step, num_epochs*len(train_loader),
                              average_train_loss, average_valid_loss))

                # checkpoint
                if best_valid_loss > average_valid_loss:
                    best_valid_loss = average_valid_loss
                    save_checkpoint(file_path + '/' + 'model.pt', model, best_valid_loss)
                    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)

    save_metrics(file_path + '/' + 'metrics.pt', train_loss_list, valid_loss_list, global_steps_list)
    print('Finished Training!')

In [20]:
model = BERT().to(device)
optimizer = optim.Adam(model.parameters(), lr=2e-5)

train(model=model, optimizer=optimizer)

Some weights of the model checkpoint at bert-base-chinese were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

RuntimeError: cuda runtime error (59) : device-side assert triggered at C:/w/1/s/windows/pytorch/aten/src\THC/generic/THCTensorMath.cu:26

In [ ]:
train_loss_list, valid_loss_list, global_steps_list = load_metrics(destination_folder + '/metrics.pt')
plt.plot(global_steps_list, train_loss_list, label='Train')
plt.plot(global_steps_list, valid_loss_list, label='Valid')
plt.xlabel('Global Steps')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Evaluation Function

def evaluate(model, test_loader):
    y_pred = []
    y_true = []

    model.eval()
    with torch.no_grad():
        for (labels, title, text, titletext), _ in test_loader:

                labels = labels.type(torch.LongTensor)
                labels = labels.to(device)
                titletext = titletext.type(torch.LongTensor)
                titletext = titletext.to(device)
                output = model(titletext, labels)

                _, output = output
                y_pred.extend(torch.argmax(output, 1).tolist())
                y_true.extend(labels.tolist())

    print('Classification Report:')
    print(classification_report(y_true, y_pred, labels=[1,0], digits=4))

    cm = confusion_matrix(y_true, y_pred, labels=[1,0])
    ax= plt.subplot()
    sns.heatmap(cm, annot=True, ax = ax, cmap='Blues', fmt="d")

    ax.set_title('Confusion Matrix')

    ax.set_xlabel('Predicted Labels')
    ax.set_ylabel('True Labels')

    ax.xaxis.set_ticklabels(['FAKE', 'REAL'])
    ax.yaxis.set_ticklabels(['FAKE', 'REAL'])

In [ ]:
best_model = BERT().to(device)

load_checkpoint(destination_folder + '/model.pt', best_model)

evaluate(best_model, test_iter)